In [2]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
import pickle
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
to_categorical = tf.keras.utils.to_categorical

In [3]:
path = os.path.join("Data/face_mask_recognition/experiements/data")
category = ["with_mask", "without_mask"]

In [4]:
training_data = []

for i in os.listdir(path):
    label = category.index(i)
    new_path = os.path.join(path, i)
    for j in os.listdir(new_path):
        img_to_array = cv2.imread(os.path.join(new_path, j), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_to_array, (100, 100))
        training_data.append([new_array, label])


In [5]:
print(len(training_data))

1376


In [6]:
X = []
y = []

for features, labels in training_data:
    X.append(features)
    y.append(labels)
    
y = to_categorical(y)

In [9]:
print(y[1])

[0. 1.]


In [1]:
print(y.shape)

NameError: name 'y' is not defined

In [10]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=10)

In [11]:
# ingat reshape dulu baru normalisasi
X_train = np.array(X_train).reshape(-1,100,100,1)
X_train = X_train/255.0

In [13]:
print(y_train[1])

[0. 1.]


In [16]:
model=tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(200,(3,3),input_shape=(100,100,1), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(100,(3,3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(50,activation='relu'))
model.add(tf.keras.layers.Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 52900)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 52900)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)               

In [18]:
from tensorflow.keras.callbacks import TensorBoard

NAME = "Face_mask_recognition"
tboard_log_dir = os.path.join("tensorboard",NAME)
tensorboard = TensorBoard(log_dir = tboard_log_dir)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("acc") > 0.99):
            print(" Reached 99% Accuracy")
            self.model.stop_training = True
            
callbacks = myCallback()

In [19]:
model.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs=30, callbacks=[callbacks])

Train on 880 samples, validate on 220 samples
Epoch 1/30
880/880 [==============================] - 91s 103ms/sample - loss: 0.7366 - acc: 0.5284 - val_loss: 0.6925 - val_acc: 0.4727
Epoch 2/30
880/880 [==============================] - 67s 76ms/sample - loss: 0.6729 - acc: 0.5477 - val_loss: 0.6252 - val_acc: 0.7455
Epoch 3/30
880/880 [==============================] - 68s 77ms/sample - loss: 0.6435 - acc: 0.6250 - val_loss: 0.6448 - val_acc: 0.5182
Epoch 4/30
880/880 [==============================] - 64s 73ms/sample - loss: 0.5568 - acc: 0.6977 - val_loss: 0.5431 - val_acc: 0.6864
Epoch 5/30
880/880 [==============================] - 67s 76ms/sample - loss: 0.4194 - acc: 0.8102 - val_loss: 0.4383 - val_acc: 0.7636
Epoch 6/30
880/880 [==============================] - 65s 74ms/sample - loss: 0.3650 - acc: 0.8295 - val_loss: 0.3309 - val_acc: 0.8955
Epoch 7/30
880/880 [==============================] - 63s 72ms/sample - loss: 0.2843 - acc: 0.8807 - val_loss: 0.2916 - val_acc: 0.8818
E

In [20]:
model.save("face_mask_trained.h5")

In [21]:
new_model = tf.keras.models.load_model("face_mask_trained.h5")
new_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 52900)             0         
________________

In [28]:
X_test = np.array(X_test).reshape(-1,100,100,1)
X_test = X_test/255.0

haha = model.evaluate(X_test, y_test)

276/276 [==============================] - 4s 15ms/sample - loss: 0.2312 - acc: 0.9239
